In [1]:
import pandas as pd
import numpy as np
import re
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.io as pio

init_notebook_mode(connected=True)
plotly.__version__

import os
import pickle

from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model  import LogisticRegression

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import train_test_split

from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostClassifier

from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin

In [2]:
tweets = pd.read_pickle('../datasets/tweetsProcesados_KM.pkl')

In [3]:
scraped = pd.concat([pd.read_pickle('../datasets/TweetsScrapedUsers_k1.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_k2.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_k3.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_m1.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_m2.pkl'),\
                     pd.read_pickle('../datasets/TweetsScrapedUsers_m3.pkl')])
original = pd.concat([pd.read_pickle('../datasets/Users_K.pkl'),\
                 pd.read_pickle('../datasets/Users_k2.pkl'),\
                 pd.read_pickle('../datasets/Users_k3.pkl'),\
                 pd.read_pickle('../datasets/Users_m1.pkl'),\
                 pd.read_pickle('../datasets/Users_m2.pkl'),\
                 pd.read_pickle('../datasets/Users_m3.pkl')])
users = original[original.id.isin(scraped.user_id)] 
users = users[pd.DataFrame(users.id.astype(str)).id.isin(tweets.user_id)]
users = users[users.id.isin(users.id.unique())]

In [4]:
#tweets = pd.read_pickle('../datasets/tweetsProcesados_KM.pkl')
#users = pd.read_pickle('../datasets/UsersDataset.pkl')

tweets = tweets[tweets.tweetsCount.apply(int) > 50]

users.index = users.index.map(int).values
tweets.index = tweets.index.map(int).values

print(len(users))
print(len(tweets))
display(users.head(2))
display(tweets.head(2))

794
756


,id,screen_name,complete_name,tweet_count,description,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
347513688,347513688,cinedelabestia,El cine de La Bestia,14046,Cine y televisión bestiales.,Buenos Aires,False,False,0,2011-08-01 00:00:00,2019-06-29 23:15:54.650537,21379,2368,543,False
470238248,470238248,14capo,HUGO,32185,Hacia la Victoria Siempre ......,Por Ahí,False,False,0,2012-01-01 00:00:00,2019-07-12 23:36:30.289443,16309,2511,3996,False


,user_id,tweetsCount,semanas,lun,mar,mie,jue,vie,sab,dom,...,num_mentions_mean,num_mentions_median,num_mentions_std,num_mentions_fq,num_mentions_tq,reply_count_mean,reply_count_median,reply_count_std,reply_count_fq,reply_count_tq
1001242472738156547,1001242472738156547,758,27,3.8,4.428571,10.428571,7.789474,4.380952,4.2,3.833333,...,4274.193548387097,0.0,6269.475295633572,0.0,10000.0,241.93548387096774,0.0,1542.7299619963394,0.0,0.0
1001522970580398082,1001522970580398082,614,14,9.0,9.363636,9.583333,7.833333,7.5,5.0,4.818182,...,0.0,0.0,0.0,0.0,0.0,11486.486486486487,0.0,33306.75535378815,0.0,10000.0


In [5]:
completeDf = tweets.join(users, how='inner')
completeDf.head()

,user_id,tweetsCount,semanas,lun,mar,mie,jue,vie,sab,dom,...,location,birthdate,url,listed_count,created_at,crawled_at,favs_count,followers_count,following_count,verified
5385972,5385972,809,13,12.181818,11.909091,9.727273,11.636364,11.416667,9.818182,6.4,...,La Plata,12 de septiembre de 1986,facebook.com/jerogi,1,2007-04-01 00:00:00,2019-07-12 23:36:41.591825,13257,4494,3721,False
5966992,5966992,681,20,4.4375,5.578947,5.352941,7.529412,4.052632,7.210526,4.176471,...,Argentina 🇦🇷,False,facebook.com/MarcelinoMAlva…,3,2007-05-01 00:00:00,2019-07-12 23:37:19.081652,42535,2412,1789,False
11850572,11850572,804,62,3.025,2.367347,2.361702,3.423077,2.960784,2.1,1.653846,...,Buenos Aires,False,facebook.com/albertodaniel.…,3,2008-01-01 00:00:00,2019-07-12 23:37:03.379724,1830,10932,488,False
33324953,33324953,748,17,6.538462,8.857143,7.285714,8.8,9.5,7.6,4.142857,...,False,False,karinabanfi.com,6,2009-04-01 00:00:00,2019-07-12 23:37:48.074707,47894,16088,1073,True
33681912,33681912,752,114,2.916667,2.02381,2.172414,2.26087,1.962264,2.133333,2.552632,...,False,False,False,0,2009-04-01 00:00:00,2019-07-12 23:38:13.814767,3159,226,729,False


In [6]:
#crear clase
class twDataTransform(TransformerMixin):
    def fit(self, X, Y):
        return self
    def transform(self, X, Y=None):
        df = X
        try:
            df.loc[:, 'crawled_at'] = pd.DatetimeIndex(pd.to_datetime(df.crawled_at, utc=None)).tz_convert(tz='UTC')
            df.loc[:, 'created_at'] = pd.DatetimeIndex(pd.to_datetime(df.created_at, utc=None)).tz_convert(tz='UTC')
        except:
            try:
                df.loc[:, 'crawled_at'] = pd.DatetimeIndex(pd.to_datetime(df.crawled_at, utc=None)).tz_localize(tz='UTC')
                df.loc[:, 'created_at'] = pd.DatetimeIndex(pd.to_datetime(df.created_at, utc=None)).tz_localize(tz='UTC')
            except:
                df.loc[:, 'crawled_at'] = pd.DatetimeIndex(pd.to_datetime(df.crawled_at, utc=None)).tz_convert(tz='UTC')
                df.loc[:, 'created_at'] = pd.DatetimeIndex(pd.to_datetime(df.created_at, utc=None)).tz_convert(tz='UTC')
        account_life = df.crawled_at - df.created_at
        account_life_meses = np.round(account_life.dt.days / 30)
        account_life_meses = account_life_meses.replace(0, 1)
        account_life_semanas = np.round(account_life.dt.days / 7)
        account_life_semanas = account_life_semanas.replace(0, 1)
        numValues = ['tweet_count', 'listed_count', 'followers_count', 'following_count']
        for i in numValues:
            colName_mes = i+'_mes'
            df.loc[:,colName_mes] = df[i].astype(float) / account_life_meses

            colName_semana = i+'_semana'
            df.loc[:,colName_semana] = df[i].astype(float) / account_life_semanas
        columnas = np.concatenate(
            [[x+'_mes'] for x in numValues]
        )
        columnasTweets = np.concatenate(
            [
                [x+'_mean',x+'_median', x+'_std', x+'_fq', x+'_tq'] for x in
                ['times','timesRT','timesUser','favorite_count','retweet_count','num_mentions','reply_count']
            ]
        )
        columnas = np.concatenate([
            ['semanas', 'lun', 'mar', 'mie', 'jue', 'vie', 'sab', 'dom'],columnas,columnasTweets
        ])
        df = df[columnas]
        df = df.fillna(0)
        df = df.applymap(lambda x: np.round(float(x), 10) if np.isfinite(float(x)) else np.nan)
        df = df.fillna(0)
        return df
    
#load model
import pickle
with open('../modelo/modeloCompleto.pkl', 'rb') as m_pred:
    model = pickle.load(m_pred)

In [7]:
predicciones = model.predict_proba(completeDf)
predicciones = pd.DataFrame(predicciones)
predicciones.index = completeDf.index
predicciones.columns = ['Prob_Human', 'Prob_Bot']
usuarios_prob = pd.concat([predicciones['Prob_Bot'], completeDf], axis=1)

In [8]:
yval = []
xval = np.linspace(0,1, 300)
for i in xval:
    cantUsuarios = len(usuarios_prob[usuarios_prob['Prob_Bot']>i])
    yval.append(cantUsuarios)
trace1 = go.Scatter(
    name='Inscriptos',
    x=xval,
    y=yval,
#     line={
#         'shape': 'spline',
#         'smoothing': 0.7,
#     }
)
layout = go.Layout(
    title=go.layout.Title(
        text='Cantidad bots',
        xref='paper',
        x=0
    ),
)
fig = go.Figure(data=[trace1], layout=layout)
print(len(usuarios_prob))
iplot(fig)

761


In [9]:
probaFilter = .7
print(len(usuarios_prob[usuarios_prob.Prob_Bot > probaFilter]) / len(usuarios_prob) * 100)
usuarios_prob[usuarios_prob.Prob_Bot > probaFilter].sort_values('Prob_Bot', ascending=False)[['Prob_Bot','screen_name','complete_name','description']]

4.993429697766097


,Prob_Bot,screen_name,complete_name,description
3042848837,0.979306,Pcia_Noticias,Provincia Noticias,Síntesis Informativa de la Provincia de Buenos...
762062108674711552,0.965068,elcorreoradio,El Correo Radio,|Magazine de noticias| • |Lunes a Viernes: 7:3...
199464393,0.923720,TrendsRosario,Trendsmap Rosario,En tiempo real las tendencias de Rosario
3434951685,0.906299,QestionPolitica,Cuestión Política,Noticias políticas de la cuarta sección electo...
1006342696838057991,0.905746,elagoradigital,El Ágora,
1121417139863326720,0.901481,Xqestendencia1,por qué carajo es tendencia?,entérate al toque porque algo es tendencia en ...
1103227572496879616,0.886361,soledad_urruti,Soledad Urruti,Vive y deja vivir
532911351,0.878716,enSEMANAecg,enSEMANA (El Correo Gráfico),enSEMANA es ahora El Correo Gráfico. Visite ht...
1121853703596670978,0.873409,verde_pantera,Pantera Verde 💚 💚,"Humor político nacional, popular, feminista y ..."
1037852333923819520,0.870631,NoraAlbornoz8,CFK No me arrepiento de este AMOR,


In [10]:
# Drop a un archivo excel para procesar
import openpyxl
usuarios_prob[usuarios_prob.Prob_Bot > probaFilter].sort_values('Prob_Bot', ascending=False)[['Prob_Bot','screen_name','complete_name','tweet_count','description']]\
.to_excel('../datasets/predictedbots_km.xlsx',index=False)